In [ ]:
# Setup

from herbie.core import Herbie
from herbie import FastHerbie
import pandas as pd
import numpy as np

In [ ]:
# Read in list of RAWS Stations
df = pd.read_csv("raws_stations_WA.csv")

# Filter to those with complete fmda data
df = df[(df[['air_temp', 'relative_humidity', 'precip_accum',
       'fuel_moisture', 'wind_speed', 'solar_radiation']]==1).sum(axis=1)==6]

# Get list of coords
points = list(df[["lon","lat"]].itertuples(index=False,name=None))
names = np.unique(df['STID'])

print(df.shape)

## FastHerbie for Multiple Times

In [ ]:
# Create a range of dates, June 1 to June 30
dates = pd.date_range(
    start="2022-06-01 00:00",
    periods=24*30, # 1 month of data
    freq="1H",
)

In [ ]:
# Make FastHerbie Object.
fh = FastHerbie(dates, model="hrrr")

In [ ]:
# 1 month of observations
print(len(fh.objects))
print(24*30)

Get data from a Single Station Location

In [ ]:
df.iloc[0]

In [ ]:
ds = fh.xarray("(?:TMP|RH):2 m", remove_grib=True)
dsi = ds.herbie.nearest_points(
    points=points[0],
    names=names[0]
)

In [ ]:
import pickle
with open('hrrr_wa_BMFW1_0623.pickle', 'wb') as handle:
    pickle.dump(dsi, handle, protocol=pickle.HIGHEST_PROTOCOL)